# Lesson 1 project: Finding Higgs decays

## Physics background

The particle-search problem may be familiar to some of you—this background section is provided in case it isn't. If you know about this stuff already, you can skip to the next section.

The general problem is that most of the particles we want to study are invisible to our instruments, but if we can observe the particles they decay into, we can reconstruct the original particles.

For example, ${K^0}_S$ is a particle with no charge and a relatively short lifetime. It decays after $10^{-10}$ seconds, which is about 3 centimeters of flight close to the speed of light. Uncharged particles do not leave traces in tracking detectors, which work by collecting ionized gas from charged particles. Fortunately, ${K^0}_S$ often decays into two charged particles, $\pi^+$ and $\pi^-$. In a tracking detector, the $\pi^+$ and $\pi^-$ seem to come out of nowhere.

On the left, we see a ${K^0}_S \to \pi^+ \pi^-$ in a bubble chamber (1960's), and on the right, we see the same decay in the CMS silicon tracker (2010's). In both cases, the vertex where the ${K^0}_S$ was created is also visible.

<center>
<img src="../img/kshort-1.png" style="height: 400px; margin-right: 20px"><img src="../img/kshort-2.png" style="height: 400px">
</center>

In a tracking detector, we can fully measure the momentum of charged particles, and since energy and momentum are conserved in decays, we know that the sum of energy and momentum of the decay products ($\pi^+$ and $\pi^-$) are the energy and momentum of the particle that decayed (${K^0}_S$). Using

$$p = \sqrt{{p_x}^2 + {p_y}^2 + {p_z}^2}$$

$$m = \sqrt{E^2 - p^2}$$

we can reconstruct the mass $m$ of the particle that decayed.

The problem is identifying which two tracks are the decay products of the specific particle we're interested in. A collision event produces many particles and many tracks.

<center>
<img src="../img/090324_ALICE-hirez.jpg" style="height: 400px">
</center>

Luckily, for most pairs of those tracks, the reconstructed mass is nowhere near the true mass of the particle. When we iterate over all pairs of tracks, compute the reconstructed masses, and plot a histogram of them, we usually see something like

<center>
<img src="../img/kshort-3.png" style="height: 500px">
</center>

The peak consists of true ${K^0}_S \to \pi^+ \pi^-$ decays, and it is not perfectly narrow because of detector resolution and quantum mechanics. (Very short lived particles have wide mass distributions.) The background under the peak consists of random track pairs that were not ${K^0}_S \to \pi^+ \pi^-$ decays.

In this exercise, you will be looking for Higgs decays, which have two stages:

$$H \to ZZ$$

$$Z \to e^+e^- \mbox{\hspace{0.25 cm}or\hspace{0.25 cm}} Z \to \mu^+\mu^-$$

<center>
<img src="../img/higgs-to-four-leptons-diagram.png" style="height: 400px">
</center>

Our detectors can distinguish charge ($e^+$ versus $e^-$) and flavor ($e$ versus $\mu$), but a collision event may have more electrons or muons than the ones that came from Higgs or Z decays. The challenge will be to iterate through these collections to reconstruct Higgs candidates without double-counting or under-counting.

## Getting data, building objects

Since this is a new notebook, we need to import the packages we'll be using and load the data.

In [ ]:
import json

import numpy as np
import vector

dataset = json.load(open("../data/SMHiggsToZZTo4L.json"))

def to_vector(particle):
    return vector.obj(
        pt=particle["pt"],
        eta=particle["eta"],
        phi=particle["phi"],
        mass=particle["mass"],
    )

Instead of separate classes for electrons and muons, let's make one class with a `flavor` attribute.

This class also has an `__add__` method to enable `particle1 + particle2` and a `mass` property that computes the mass from its energy and momentum.

In [ ]:
class Particle:
    def __init__(self, E, px, py, pz, charge, flavor):
        self.E = E
        self.px = px
        self.py = py
        self.pz = pz
        self.charge = charge
        self.flavor = flavor

    def __repr__(self):
        return f"<{type(self).__name__} E={self.E} px={self.px} py={self.py} pz={self.pz} charge={self.charge} flavor={self.flavor!r}>"

    # the '__add__' method gives meaning to 'particle + particle'
    def __add__(self, other):
        return Particle(
            self.E + other.E,
            self.px + other.px,
            self.py + other.py,
            self.pz + other.pz,
            self.charge + other.charge,
            "none"
            if self.charge + other.charge == 0
            else f"{self.flavor}-{other.flavor}",
        )

    # '@property' means we can call this method without parentheses, as though it were an attribute
    @property
    def mass(self):
        return vector.obj(E=self.E, px=self.px, py=self.py, pz=self.pz).mass

In [ ]:
leptons = []  # a "lepton" is an electron or a muon, distinguished by its "flavor"

event = dataset[96]  # a nice event with 3 electrons and 3 muons

for particle in event["electron"]:
    v = to_vector(particle)
    leptons.append(Particle(v.E, v.px, v.py, v.pz, particle["charge"], "electron"))

for particle in event["muon"]:
    v = to_vector(particle)
    leptons.append(Particle(v.E, v.px, v.py, v.pz, particle["charge"], "muon"))

leptons

The `__add__` function lets us combine two particles to get what their parent `Particle` would be _if_ those particles come from the same decay.

In [ ]:
z_boson = leptons[0] + leptons[2]
z_boson

In [ ]:
z_boson.mass

In the following, we use `enumerate`:

```python
list(enumerate(["a", "b", "c"])) == [(0, "a"), (1, "b"), (2, "c")]
```

and `i < j` to ensure that if we include lepton pair $(i, j)$, we don't also include $(j, i)$.

In [ ]:
z_candidates_step0 = []
for i, lepton_i in enumerate(leptons):
    for j, lepton_j in enumerate(leptons):
        if i < j:
            z_candidates_step0.append({"index": [i, j], "Z boson": lepton_i + lepton_j})

There are 15 lepton pairs in this event.

In [ ]:
z_candidates_step0

<br><br><br><br><br>

## Exercise part 1

Z bosons always decay into particles of opposite charge and identical flavor. Reduce the set of candidates by excluding ones with the wrong properties.

Replace the `if ...:` with an `if` statement that selects lepton pairs with the right properties.

In [ ]:
z_candidates_step1 = []
for candidate in z_candidates_step0:
    i, j = candidate["index"]
    z_boson = candidate["Z boson"]
    if ...:
        z_candidates_step1.append(candidate)

The number of Z boson candidates you have left should be 8.

In [ ]:
len(z_candidates_step1)

Print the masses of these Z boson candidates. They should be

| Z mass (GeV/$c^2$) |
|-----------------:|
| 94.65200565609616 |
| 62.03397488944119 |
|  3.41705043610203 |
|  3.08773290949845 |
| 45.69023328291948 |
|  3.66225837801406 |
| 26.45024522236556 |
|  3.27373703909123 |

<br><br><br><br><br>

## Exercise part 2

The Higgs boson decays into two Z bosons. The only constraint here is that a lepton from one Z decay can't also be a lepton from the other Z decay.

Replace the `if ...:` with an `if` statement that rejects double-counted indexes.

In [ ]:
higgs_candidates_step1 = []

for z_index1, z_candidate1 in enumerate(z_candidates_step1):
    for z_index2, z_candidate2 in enumerate(z_candidates_step1):
        if z_index1 < z_index2:
            lepton_i1, lepton_j1 = z_candidate1["index"]
            lepton_i2, lepton_j2 = z_candidate2["index"]
            z_boson1 = z_candidate1["Z boson"]
            z_boson2 = z_candidate2["Z boson"]

            if ...:
                higgs_candidates_step1.append(
                    {
                        "index": [lepton_i1, lepton_j1, lepton_i2, lepton_j2],
                        "H boson": z_boson1 + z_boson2,
                        "Z bosons": [z_boson1, z_boson2],
                    }
                )

The number of Higgs candidates should be 12.

In [ ]:
len(higgs_candidates_step1)

That's still too many. Each of these events were simulated with exactly one Higgs boson.

To see what the matter might be, print out the indexes and the masses of these candidates. It should be

| index | H mass (GeV/$c^2$) |
|:-----:|------------------:|
| [0, 2, 1, 3] | 118.2598039063236 |
| [0, 2, 3, 4] | 129.0346159691587 |
| [0, 2, 3, 5] | 118.8311777089632 |
| [0, 3, 1, 2] | 118.2598039063236 |
| [0, 3, 2, 4] | 129.0346159691587 |
| [0, 3, 2, 5] | 118.8311777089631 |
| [1, 2, 3, 4] |  56.1098916972126 |
| [1, 2, 3, 5] |  12.7507340718563 |
| [1, 3, 2, 4] |  56.1098916972126 |
| [1, 3, 2, 5] |  12.7507340718557 |
| [2, 4, 3, 5] |  56.3856286987971 |
| [2, 5, 3, 4] |  56.3856286987971 |

Even though each candidate avoids double-counting within itself, the same combination of four indexes can be found among the candidates. We want only one of each.

Let's collect these Higgs candidates by unique sets of indexes. The `sorted` function sorts a list, and `tuple` makes it possible to use them as keys in a dict.

In [ ]:
higgs_candidates_step2 = {}

for higgs_candidate in higgs_candidates_step1:
    combination = tuple(sorted(higgs_candidate["index"]))

    if combination not in higgs_candidates_step2:
        higgs_candidates_step2[combination] = []

    higgs_candidates_step2[combination].append(higgs_candidate)

This `higgs_candidates_step2` has deep structure:

  * Keys are sets combinations of lepton indexes, without regard for their original order.
  * Values are a list of decay trees.
    - Each element of that list has a candidate Higgs mass and two candidate Z masses.

In [ ]:
for combination in higgs_candidates_step2:
    print(combination)
    for higgs_candidate in higgs_candidates_step2[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        print(
            "    Higgs:",
            higgs_candidate["H boson"].mass,
            "Z:",
            z_boson1.mass,
            z_boson2.mass,
        )

<br><br><br><br><br>

## Exercise part 3

One of the selections that the 2012 Higgs discovery analysis applied was:

  * 12 GeV/$c^2$ < smallest Z mass < 120 GeV/$c^2$
  * 40 GeV/$c^2$ < largest Z mass < 120 GeV/$c^2$

because this is expected of real Higgs decays.

Apply the Z mass constraint to these Higgs candidates.

In [ ]:
higgs_candidates_step3 = {}

for combination in higgs_candidates_step2:
    higgs_candidates_step3[combination] = []

    for higgs_candidate in higgs_candidates_step2[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        smallest_z_mass = ...
        largest_z_mass = ...

        if ...:
            higgs_candidates_step3[combination].append(higgs_candidate)

In the end,

In [ ]:
for combination in higgs_candidates_step3:
    print(combination)
    for higgs_candidate in higgs_candidates_step3[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        print(
            "    Higgs:",
            higgs_candidate["H boson"].mass,
            "Z:",
            z_boson1.mass,
            z_boson2.mass,
        )

should be

```
(0, 1, 2, 3)
(0, 2, 3, 4)
    Higgs: 129.03461596915875 Z: 94.65200565609616 26.45024522236556
    Higgs: 129.03461596915875 Z: 62.03397488944119 45.69023328291948
(0, 2, 3, 5)
(1, 2, 3, 4)
(1, 2, 3, 5)
(2, 3, 4, 5)
```

In other words, there's only one combination of 4 leptons for this event, but that one Higgs might be divided up among Z bosons in two different ways.

<br><br><br>

This aspect of particle physics—the fact that observed particles can be associated with a decay tree in multiple ways—is known as "combinatorics."

<br>

Complex, nested data structures and combinatorics are essential aspects of particle physics analysis.